In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib   


In [2]:
train_dir = "C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset/train"
test_dir = "C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset/test"

In [3]:
input_shape = (128, 128, 3)
batch_size = 32

In [4]:
data_generator = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [6]:
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [7]:
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for KNN
train_labels = train_generator.classes

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 15s 92ms/step


In [9]:
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 127ms/step


In [10]:
knn_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust n_neighbors as needed
knn_classifier.fit(train_features, train_labels)

KNeighborsClassifier()

In [11]:
test_predictions = knn_classifier.predict(test_features)

In [12]:
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 59.03%


In [13]:
model.save('cnn_Feature_Extractor2.h5')

In [14]:
joblib.dump(knn_classifier, 'knn_classifier.pkl')

['knn_classifier.pkl']

In [16]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image

cnn_model = load_model('cnn_Feature_Extractor2.h5')
svm_classifier = joblib.load('svm_classifier.pkl')

def classify_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))  
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  
    img_array /= 255.0  
    
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  

    prediction = knn_classifier.predict(features)
    
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented',2: 'NonDemented',3:'VeryMildDemented'} 
    result = class_labels[prediction[0]]
    return result

img_path = r"C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset/test/NonDemented/32 (7).jpg"
result = classify_image(img_path)
print(f"The image is classified as: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
The image is classified as: NonDemented
